In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!nvidia-smi

Tue Aug 31 04:23:02 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 470.57.02    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla P100-PCIE...  Off  | 00000000:00:04.0 Off |                    0 |
| N/A   38C    P0    26W / 250W |      0MiB / 16280MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
!pip install transformers==3.5.1
!pip install torch==1.4.0

     |████████████████████████████████| 1.3 MB 9.0 MB/s 
     |████████████████████████████████| 895 kB 66.1 MB/s 
     |████████████████████████████████| 2.9 MB 62.4 MB/s 
     |████████████████████████████████| 1.1 MB 70.8 MB/s 
     |████████████████████████████████| 753.4 MB 7.0 kB/s 
  Attempting uninstall: torch
    Found existing installation: torch 1.9.0+cu102
    Uninstalling torch-1.9.0+cu102:
      Successfully uninstalled torch-1.9.0+cu102
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
torchvision 0.10.0+cu102 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.
torchtext 0.10.0 requires torch==1.9.0, but you have torch 1.4.0 which is incompatible.


In [ ]:
import transformers
import numpy as np
import torch
from functools import partial
from tqdm import tqdm

In [ ]:
from transformers.data.metrics.squad_metrics import compute_predictions_log_probs, compute_predictions_logits, squad_evaluate
from transformers.data.processors.squad import SquadResult, SquadV1Processor, SquadV2Processor

In [ ]:
from transformers import AutoModel, AutoTokenizer
from transformers import RobertaForQuestionAnswering, RobertaConfig, WEIGHTS_NAME

#***************TRAINING************
phobert = AutoModel.from_pretrained("vinai/phobert-base")
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = RobertaForQuestionAnswering(phobert.config).from_pretrained("vinai/phobert-base")
# phobert = AutoModel.from_pretrained("/content/drive/MyDrive/Colab Notebooks/results/final_model")
# tokenizer = AutoTokenizer.from_pretrained("/content/drive/MyDrive/Colab Notebooks/results/final_model")
# model = RobertaForQuestionAnswering(phobert.config).from_pretrained("/content/drive/MyDrive/Colab Notebooks/results/final_model")
#***********************************

#**************TESTING**************
# phobert = AutoModel.from_pretrained("vinai/phobert-base")
# tokenizer = AutoTokenizer.from_pretrained("../input/phobert/final_model")
# model = RobertaForQuestionAnswering(phobert.config).from_pretrained("../input/phobert/final_model")

Downloading:   0%|          | 0.00/557 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/543M [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/895k [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/1.14M [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embedding are fine-tuned or trained.
Some weights of the model checkpoint at vinai/phobert-base were not used when initializing RobertaForQuestionAnswering: ['lm_head.bias', 'lm_head.dense.weight', 'lm_head.dense.bias', 'lm_head.layer_norm.weight', 'lm_head.layer_norm.bias', 'lm_head.decoder.weight', 'lm_head.decoder.bias']
- This IS expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing RobertaForQuestionAnswering from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of RobertaForQuestionAnswering were not initialized from the model checkpoint at vinai/phobe

In [ ]:
processor = SquadV1Processor()

In [ ]:
train_examples = processor.get_train_examples('/content/drive/MyDrive/Colab Notebooks','train_ViQuAD_ws.json')
dev_examples = processor.get_dev_examples('/content/drive/MyDrive/Colab Notebooks', 'dev_ViQuAD_ws.json')
test_examples = processor.get_dev_examples('/content/drive/MyDrive/Colab Notebooks', 'test_ViQuAD_ws.json')

100%|██████████| 8840/8840 [00:05<00:00, 1721.88it/s]


In [ ]:
from transformers.data.processors.squad import squad_convert_examples_to_features

In [ ]:
train_features, train_dataset = squad_convert_examples_to_features(train_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 64,
                                                       max_query_length = 64,
                                                       is_training = True,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

convert squad examples to features:   0%|          | 0/18579 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  Fu

In [ ]:
dev_features, dev_dataset = squad_convert_examples_to_features(dev_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 64,
                                                       max_query_length = 64,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

convert squad examples to features:   0%|          | 0/9140 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  Fut

In [ ]:
test_features, test_dataset = squad_convert_examples_to_features(test_examples, 
                                                       tokenizer, 
                                                       max_seq_length = 256, 
                                                       doc_stride = 64,
                                                       max_query_length = 64,
                                                       is_training = False,
                                                       return_dataset = 'pt',
                                                       threads = 10
                                                       )

convert squad examples to features:   0%|          | 0/8840 [00:00<?, ?it/s]/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  FutureWarning,
/usr/local/lib/python3.7/dist-packages/transformers/tokenization_utils_base.py:1423: FutureWarning: The `max_len` attribute has been deprecated and will be removed in a future version, use `model_max_length` instead.
  Fut

In [ ]:
def to_list(tensor):
    return tensor.detach().cpu().tolist()

In [ ]:
import os
def evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features):
    eval_sampler = SequentialSampler(dev_dataset)
    eval_dataloader = DataLoader(dev_dataset, sampler=eval_sampler, batch_size=12)
    all_results = []
#     start_time = timeit.default_timer()
    for batch in tqdm(eval_dataloader, desc="Evaluating"):
        model.eval()
        batch = tuple(t.to(device) for t in batch)
        with torch.no_grad():
            inputs = {
                "input_ids": batch[0],
                "attention_mask": batch[1],
                "token_type_ids": batch[2],
            }
            example_indices = batch[3]
            outputs = model(**inputs)
        for i, example_index in enumerate(example_indices):
            eval_feature = dev_features[example_index.item()]
            unique_id = int(eval_feature.unique_id)
#             for output in outputs:
#                 print(output)
            output = [to_list(output[i]) for output in outputs]
#             output = [to_list(output) for output in outputs]
            if len(output) >= 5:
                start_logits = output[0]
                start_top_index = output[1]
                end_logits = output[2]
                end_top_index = output[3]
                cls_logits = output[4]

                result = SquadResult(
                    unique_id,
                    start_logits,
                    end_logits,
                    start_top_index=start_top_index,
                    end_top_index=end_top_index,
                    cls_logits=cls_logits,
                )
            else:
                start_logits, end_logits = output
                result = SquadResult(unique_id, start_logits, end_logits)
            all_results.append(result)
    
    output_prediction_file = os.path.join("./", "predictions_{}.json".format(""))
    output_nbest_file = os.path.join("./", "nbest_predictions_{}.json".format(""))
    output_null_log_odds_file = os.path.join("./", "null_odds_{}.json".format(""))
    predictions = compute_predictions_logits(
            dev_examples,
            dev_features,
            all_results,
            20,
            30,
            True,
            output_prediction_file,
            output_nbest_file,
            output_null_log_odds_file,
            True,
            True,
            0.0,
            tokenizer,
        )
    results = squad_evaluate(dev_examples, predictions)
    return results

In [ ]:
from torch.utils.tensorboard import SummaryWriter
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler
from transformers import AdamW, get_linear_schedule_with_warmup
from tqdm import trange, tqdm
tqdm = partial(tqdm, position=0, leave=True)

In [ ]:
device = torch.device('cuda')
model.to(device)
tb_writer = SummaryWriter()
train_sampler = RandomSampler(train_dataset)
train_dataloader = DataLoader(train_dataset, sampler=train_sampler, batch_size=12)
t_total = len(train_dataloader)


no_decay = ["bias", "LayerNorm.weight"]

optimizer_grouped_parameters = [
    {
        "params": [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)],
        "weight_decay": 0,
    },
    {"params": [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], "weight_decay": 0.0},
]
optimizer = AdamW(optimizer_grouped_parameters, lr=3e-5, eps = 1e-8)
scheduler = get_linear_schedule_with_warmup(
    optimizer, num_warmup_steps=20, num_training_steps=t_total
)


global_step = 1
epochs_trained = 0
steps_trained_in_current_epoch = 0
tr_loss, logging_loss = 0.0, 0.0

model.zero_grad()
train_iterator = trange(
    epochs_trained, int(40), desc="Epoch", disable=-1 not in [-1, 0]
)

for _ in train_iterator:
    epoch_iterator = tqdm(train_dataloader, desc="Iteration", disable=False)
    for step, batch in enumerate(epoch_iterator):
        model.train()
        batch = tuple(t.to(device) for t in batch)
        
        inputs = {
            "input_ids": batch[0],
            "attention_mask": batch[1],
            "token_type_ids": batch[2],
            "start_positions": batch[3],
            "end_positions": batch[4],
        }
        
        outputs = model(**inputs)
        loss = outputs[0]
        loss.backward()
        tr_loss += loss.item()
#         if (step + 1) % 1 == 0: #:))
        torch.nn.utils.clip_grad_norm_(model.parameters(), 1)
        optimizer.step()
        scheduler.step()
        model.zero_grad()
        global_step += 1

#         if global_step % 10 == 0:

#                 tb_writer.add_scalar("eval_{}".format(key), value, global_step)
#             tb_writer.add_scalar("lr", scheduler.get_lr()[0], global_step)
#             tb_writer.add_scalar("loss", (tr_loss - logging_loss) / 50, global_step)
#             logging_loss = tr_loss
        if global_step % 5000 == 0:
#             output_dir = os.path.join('./', "checkpoint-{}".format(global_step))
#             model_to_save = model.module if hasattr(model, "module") else model
#             model_to_save.save_pretrained(output_dir)
#             tokenizer.save_pretrained(output_dir)
#             torch.save(optimizer.state_dict(), os.path.join(output_dir, "optimizer.pt"))
#             torch.save(scheduler.state_dict(), os.path.join(output_dir, "scheduler.pt"))
            print(" global_step = %s, average loss = %s" % (global_step, tr_loss/global_step))

            
output_dir = os.path.join('./', 'final_model')
model_to_save = model.module if hasattr(model, "module") else model
model_to_save.save_pretrained(output_dir)
tokenizer.save_pretrained(output_dir)

print(" global_step = %s, average loss = %s" % (global_step, tr_loss/global_step))

results = evaluate(model, tokenizer, dev_dataset, dev_examples, dev_features)
for key, value in results.items():
    print(key, value)

Iteration:  53%|█████▎    | 1055/1972 [05:47<05:02,  3.04it/s]

 global_step = %s, average loss = %s 5000 1.357460815757513


Iteration:   7%|▋         | 139/1972 [00:45<10:03,  3.04it/s]

 global_step = %s, average loss = %s 10000 1.2213499365270137


Iteration:  61%|██████    | 1195/1972 [06:33<04:15,  3.04it/s]

...

 global_step = %s, average loss = %s 75000 1.1021183771038054


Epoch: 100%|██████████| 40/40 [7:12:20<00:00, 648.51s/it]


 global_step = %s, average loss = %s 78881 1.1012338616551907


Evaluating: 100%|██████████| 905/905 [01:33<00:00,  9.70it/s]


exact 47.410008779631255
f1 72.80744010327244
total 2278
HasAns_exact 47.410008779631255
HasAns_f1 72.80744010327244
HasAns_total 2278
best_exact 47.410008779631255
best_exact_thresh 0.0
best_f1 72.80744010327244
best_f1_thresh 0.0


In [ ]:
print("*********** Test result:")
results = evaluate(model, tokenizer, test_dataset, test_examples, test_features)
for key, value in results.items():
    print(key, value)

*********** Test result:


Evaluating: 100%|██████████| 928/928 [01:36<00:00,  9.57it/s]


exact 47.780797101449274
f1 70.60434134101155
total 2208
HasAns_exact 47.780797101449274
HasAns_f1 70.60434134101155
HasAns_total 2208
best_exact 47.780797101449274
best_exact_thresh 0.0
best_f1 70.60434134101155
best_f1_thresh 0.0


In [ ]:
!zip "/content/drive/MyDrive/Colab Notebooks/final_model_3.zip" "./final_model"

  adding: final_model/ (stored 0%)
